## Techniche - Topic Modelling

In [1]:
import pandas as pd
import numpy as np

import gensim
import gensim.corpora as corpora
from gensim.corpora import mmcorpus
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, Phraser
from gensim.models.ldamodel import LdaModel
from gensim.models import AuthorTopicModel
from gensim.test.utils import common_dictionary, datapath, temporary_file
from smart_open import smart_open

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, punkt, RegexpTokenizer, wordpunct_tokenize
from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer

import json
from pandas.io.json import json_normalize
import requests
import re
import os

from topic_model import tokenize_docs#, (TODO) Lee convert_bytes

from smart_open import smart_open

import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim

from pprint import pprint

import pyspark
import pyspark.sql.functions as F

/home/glmack_agoralytix_com/.local/lib/python3.7/site-packages/py4j/java_collections.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import (
/home/glmack_agoralytix_com/.local/lib/python3.7/site-packages/py4j/java_collections.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import (
/home/glmack_agoralytix_com/.local/lib/python3.7/site-packages/py4j/java_collections.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import (


In [2]:
# pd.set_option('display.max_colwidth', -1)
pd.options.display.max_columns = 50
pd.set_option('display.max_rows', 50)

In [3]:
np.random.seed(3)

In [ ]:
# uncomment to download stop words from nltk and language package from spacy
# nltk.download('stopwords')
# nltk.download('punkt')
# !python -m spacy download en

### Import Data

#### Import data from PatentsView API

In [ ]:
# patents endpoint
endpoint_url = 'http://www.patentsview.org/api/patents/query'

# build list of possible fields that endpoint request will return
df = pd.read_excel("data/patents_view_patents_fields.xlsx")
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
pat_fields = df.api_field_name.values.tolist()

# build query
query={"_or":[{"_text_phrase":{"patent_title":"natural language"}},{"_text_phrase":{"patent_abstract":"natural language"}}]}
fields=pat_fields
options={"per_page":2500}
sort=[{"patent_date":"desc"}]

params={'q': json.dumps(query),
        'f': json.dumps(fields),
        'o': json.dumps(options),
        's': json.dumps(sort)}

# request and results
resp = requests.get(endpoint_url, params=params)
results = resp.json()

In [4]:
# patents endpoint
endpoint_url = 'http://www.patentsview.org/api/patents/query'

# build list of possible fields that endpoint request will return
df = pd.read_excel("data/patents_view_patents_fields.xlsx")
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
pat_fields = df.api_field_name.values.tolist()

# build query
query={"_and":
        [{"_or":
            [{"_text_phrase":{"patent_title":"natural language"}}
            ,{"_text_phrase":{"patent_abstract":"natural language"}}]}
        ,{"_and":
      [{"patent_year":2016}]}]} 

fields=pat_fields
options={"per_page":2500}
sort=[{"patent_date":"desc"}]

params={'q': json.dumps(query),
        'f': json.dumps(fields),
        'o': json.dumps(options),
        's': json.dumps(sort)}

# request and results
resp = requests.get(endpoint_url, params=params)
results = resp.json()

In [2]:
# patents endpoint
endpoint_url = 'http://www.patentsview.org/api/patents/query'

# build list of possible fields that endpoint request will return
df = pd.read_excel("data/patents_view_patents_fields.xlsx")
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
pat_fields = df.api_field_name.values.tolist()

# build query
query={"patent_year":2016}
fields=pat_fields
options={"per_page":2500}
sort=[{"patent_date":"desc"}]

params={'q': json.dumps(query),
        'f': json.dumps(fields),
        'o': json.dumps(options),
        's': json.dumps(sort)}

# request and results
resp = requests.get(endpoint_url, params=params)
results = resp.json()

#### Structure data

In [3]:
# extract metadata from response
print("status code:", resp.status_code,';', "reason:", resp.reason)
total_patent_count = results["total_patent_count"]
patents_per_page = results['count']
print("total_patent_count:",total_patent_count,';', "patents_per_page:", patents_per_page)

# extract data from response
data_resp = results['patents']
# data_resp[0]
raw_df = pd.DataFrame(data_resp)
raw_df.head(3)

status code: 200 ; reason: OK
total_patent_count: 100000 ; patents_per_page: 2500


,IPCs,application_citations,applications,assignees,cited_patents,citedby_patents,cpcs,detail_desc_length,examiners,foreign_priority,...,patent_num_us_patent_citations,patent_number,patent_processing_time,patent_title,patent_type,patent_year,pct_data,rawinventors,uspcs,wipos
0,"[{'ipc_action_date': '2016-12-27', 'ipc_class'...","[{'appcit_app_number': None, 'appcit_category'...","[{'app_country': 'US', 'app_date': '2015-05-22...","[{'assignee_city': None, 'assignee_country': N...",[{'cited_patent_category': 'cited by examiner'...,"[{'citedby_patent_category': None, 'citedby_pa...","[{'cpc_category': 'inventional', 'cpc_first_se...",8938,"[{'examiner_first_name': 'Paul T', 'examiner_i...","[{'forprior_country': None, 'forprior_date': N...",...,8,9526196,585,Multi-purpose spade having detachable tip,utility,2016,"[{'pct_102_date': None, 'pct_371_date': None, ...","[{'rawinventor_first_name': 'Hyun C.', 'rawinv...","[{'uspc_first_seen_date': '2015-06-02', 'uspc_...","[{'wipo_field_id': '29', 'wipo_field_title': '..."
1,"[{'ipc_action_date': '2016-12-27', 'ipc_class'...","[{'appcit_app_number': '2014/20140166320', 'ap...","[{'app_country': 'US', 'app_date': '2012-10-24...","[{'assignee_city': 'Uberlingen', 'assignee_cou...",[{'cited_patent_category': 'cited by examiner'...,"[{'citedby_patent_category': None, 'citedby_pa...","[{'cpc_category': 'inventional', 'cpc_first_se...",12582,"[{'examiner_first_name': 'Minh', 'examiner_id'...","[{'forprior_country': 'DE', 'forprior_date': '...",...,16,9526197,1525,Force-transmitting unit,utility,2016,"[{'pct_102_date': None, 'pct_371_date': None, ...","[{'rawinventor_first_name': 'Artur', 'rawinven...","[{'uspc_first_seen_date': '2015-06-02', 'uspc_...","[{'wipo_field_id': '29', 'wipo_field_title': '..."
2,"[{'ipc_action_date': '2016-12-27', 'ipc_class'...","[{'appcit_app_number': '2002/20020081047', 'ap...","[{'app_country': 'US', 'app_date': '2013-08-21...","[{'assignee_city': 'Vaderstad', 'assignee_coun...",[{'cited_patent_category': 'cited by applicant...,[{'citedby_patent_category': 'cited by examine...,"[{'cpc_category': 'inventional', 'cpc_first_se...",21117,"[{'examiner_first_name': 'John G', 'examiner_i...","[{'forprior_country': 'SE', 'forprior_date': '...",...,10,9526198,1224,"Suspension unit for agricultural implement, ag...",utility,2016,"[{'pct_102_date': None, 'pct_371_date': None, ...","[{'rawinventor_first_name': 'Henrik', 'rawinve...","[{'uspc_first_seen_date': '2015-06-02', 'uspc_...","[{'wipo_field_id': '31', 'wipo_field_title': '..."


#### Import data from bulk download

In [ ]:
# uncomment to download TSV files containing detailed patent descriptions from PatentsView 
# !wget http://data.patentsview.org/detail-description-text/detail-desc-text-2016.tsv.zip # 2016 - 3.0 GB zipped
# !wget http://data.patentsview.org/detail-description-text/detail-desc-text-2017.tsv.zip # 2017 - 2.8 GB zipped
# !wget http://data.patentsview.org/detail-description-text/detail-desc-text-2018.tsv.zip # 2018 - 1.6 GB zipped
# !wget http://data.patentsview.org/detail-description-text/detail-desc-text-2019.tsv.zip # 2019 - 0.7 GB zipped

# !unzip files
# unzip detail-desc-text-2016.tsv.zip
# unzip detail-desc-text-2017.tsv.zip
# unzip detail-desc-text-2018.tsv.zip
# unzip detail-desc-text-2019.tsv.zip

def convert_bytes(num, suffix='B'):
    """ convert bytes int to int in aggregate units"""
    for unit in ['','K','M','G','T','P','E','Z']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

path = "data/"
with os.scandir(path) as it:
    for entry in it:
        if not entry.name.startswith('.') and entry.is_file():
            print(entry.name)

In [117]:
# inspect unzipped file sizes
convert_bytes(os.path.getsize("data/detail-desc-text-2016.tsv"))

'12.5GB'

In [121]:
convert_bytes(os.path.getsize("data/detail-desc-text-2017.tsv"))

'13.8GB'

In [122]:
convert_bytes(os.path.getsize("data/detail-desc-text-2018.tsv"))

'7.2GB'

In [123]:
convert_bytes(os.path.getsize("data/detail_desc_text_2019.tsv"))

'3.0GB'

In [4]:
# create SparkSession/SparkContext as entry point to Dataset/DataFrame API
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [78]:
from functools import reduce

In [79]:
files = ["data/detail-desc-text-2016.tsv", "data/detail-desc-text-2017.tsv", 
         "data/detail-desc-text-2018.tsv", "data/detail-desc-text-2019.tsv"]

In [83]:
# df = reduce(lambda x,y: x.unionAll(y), 
#             [spark.read.format('csv')
#                        .load(f, header="true", inferSchema="true") 
#              for f in files])
# df.show()

In [6]:
df_2018 = (spark.read
               .format("csv")
               .option("delimiter", "\t")
               .option('inferSchema', "true")
               .load("data/detail-desc-text-2018.tsv")
               .write
               .format("parquet")
               .save("df_2018.parquet"))

AnalysisException: 'path file:/home/glmack_agoralytix_com/techniche/df_2018.parquet already exists.;'

In [59]:
df_2018.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: integer (nullable = true)



In [64]:
schema = StructType([
            StructField("_c0", IntegerType(), True),
            StructField("_c1", StringType(), True),
            StructField("_c2", IntegerType(), True)])

NameError: name 'StructType' is not defined

In [114]:
# query file directly with SQL

query = """
SELECT _c0 FROM parquet.`data/df_2018.parquet` WHERE _c1 LIKE 'natural language' LIMIT 100
"""

df_2018_nl = spark.sql(query)

df_2018_nl.head(3)

In [6]:
# 160249 rows in 2018 dataset
df_2018.count()

160249

In [8]:
df_2018.columns

['_c0', '_c1', '_c2']

In [9]:
df_2018.explain()

== Physical Plan ==
*(1) FileScan csv [_c0#10,_c1#11,_c2#12] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/glmack_agoralytix_com/techniche/data/detail-desc-text-2018.tsv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<_c0:string,_c1:string,_c2:string>


In [11]:
df_2018.describe().show()

+-------+--------------------+--------------------+------------------+
|summary|                 _c0|                 _c1|               _c2|
+-------+--------------------+--------------------+------------------+
|  count|              160249|              160249|             37278|
|   mean|1.0055075034692628E7|                null|47672.901443210474|
| stddev|   80038.35207688982|                null|53306.603326141594|
|    min|            10000000|#System-Minimal# ...|             10000|
|    max|             RE47183|•Microphone• Here...|             99984|
+-------+--------------------+--------------------+------------------+



In [12]:
df_2018.dtypes

[('_c0', 'string'), ('_c1', 'string'), ('_c2', 'string')]

In [ ]:
df_171819 = df_2017.union(df_2018).union(df_2019)

In [ ]:
df_171819.count()

In [ ]:
df_171819.head(3)

In [ ]:
df_2018.head(3)

In [50]:
counts = df_2018.agg(F.countDistinct('_c0'))
counts

DataFrame[count(DISTINCT _c0): bigint]

In [ ]:
reviews_df.createOrReplaceTempView('reviews')

In [ ]:
output = spark.sql(query)

In [ ]:
show(output, n=1000)

In [ ]:
results = spark.sql(
  "SELECT * FROM people")
names = results.map(lambda p: p.name)

In [87]:
df.rdd.isEmpty()

NameError: name 'df' is not defined

In [88]:
df = (spark.read
            .load("data/*.parquet")
            .write
            .format("parquet")
            .save("df.parquet"))

Py4JJavaError: An error occurred while calling o653.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.sc

In [67]:
df_2019 = (spark.read
               .format("csv")
               .option("delimiter", "\t")
               .option('inferSchema', "true")
               .load("data/detail-desc-text-2018.tsv")
               .write
               .format("parquet")
               .save("df_2019.parquet"))

Py4JJavaError: An error occurred while calling o245.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.sc

In [46]:
df_2019.head(3)

[Row(_c0='9872055', _c1="The following description is merely exemplary in nature and is not intended to limit the present disclosure, application, or uses. For purposes of clarity, the same reference numbers will be used in the drawings to identify similar elements. As used herein, the term module refers to an application specific integrated circuit (ASIC), an electronic circuit, a general purpose computing device, a processor (shared, dedicated, or group) and memory that execute one or more software or firmware programs, a combinational logic circuit, and/or other suitable components that provide the described functionality. As used herein, the phrase “at least one of A, B, and C” should be construed to mean a logical (A or B or C), using a non-exclusive logical OR. It should be understood that steps within a method may be executed in different order without altering the principles of the present disclosure. The teachings of the present disclosure can be implemented in a system for co

In [ ]:
type(df_2018)

In [70]:
df_2016.head(3)

NameError: name 'df_2016' is not defined

In [73]:
df_2016 = (spark.read.format("csv")
               .option("delimiter", "\t")
               .option("header", "true")
               .option('inferSchema', "true")
               .load("data/detail-desc-text-2016.tsv")
               .write
               .format("parquet")
               .save("data/df_2016.parquet"))

Py4JJavaError: An error occurred while calling o457.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.sc

In [ ]:
df_2016.head(3)

In [ ]:
df = (spark.read.format("csv")
           .option("delimiter", ",")
           .infer
           .load("data/df.csv"))

In [84]:
df_2017 = (spark.read.format("csv")
               .option("delimiter", "\t")
               .option("header", "true")
               .option('inferSchema', "true")
               .load("data/detail-desc-text-2017.tsv")
               .write
               .format("parquet")
               .save("df_2017.parquet"))

AnalysisException: 'path file:/home/glmack_agoralytix_com/techniche/df_2017.parquet already exists.;'

In [ ]:
df_2018 = spark.read.parquet("data/df_2018.parquet")

In [ ]:
df_2018.head(2)

In [ ]:
df_2018.persist()

In [ ]:
df_2018.take(2)

In [ ]:
df_2018.toPandas()

#### Subset dataframe

In [ ]:
# subset dataframe - comment/uncomment to include fields
df = raw_df[['patent_number', 
         'patent_date', 
         'patent_title',
         'patent_abstract', 
         'patent_firstnamed_assignee_id',
         'patent_firstnamed_assignee_location_id',
         'patent_firstnamed_assignee_latitude',
         'patent_firstnamed_assignee_longitude',
         'patent_firstnamed_assignee_city',
         'patent_firstnamed_assignee_state',
         'patent_firstnamed_assignee_country', 
         'patent_firstnamed_inventor_id',
         'patent_firstnamed_inventor_location_id',
         'patent_firstnamed_inventor_latitude',
         'patent_firstnamed_inventor_longitude',
         'patent_firstnamed_inventor_city',
         'patent_firstnamed_inventor_state',
         'patent_firstnamed_inventor_country',
         'patent_year', 
         'patent_type', 
         'patent_kind',
         'inventors'
#          'patent_processing_time', 
#          'patent_num_us_application_citations', 
#          'patent_num_us_patent_citations', 
#          'patent_num_foreign_citations', 
#          'patent_num_combined_citations', 
#          'patent_num_claims', 
#          'patent_num_cited_by_us_patents',
#          'detail_desc_length'
            ]]
df.head(3)

#### Explore data

In [ ]:
# 561 different assignees
len(df.patent_firstnamed_assignee_id.unique())

#### Create new column

In [ ]:
# create new column that combines the patent title and the patent abstract columns into a single string
df['patent_title_abstract'] = df.patent_title + ' ' + df.patent_abstract
df.patent_title_abstract.head(3)

In [ ]:
#### TODO (Lee) Partition data into train and test sets

In [ ]:
len(df)

In [ ]:
df.head(3)

In [ ]:
len(df.patent_number.unique())

In [ ]:
df.sort_values(by=['patent_date'])

In [ ]:
print(df.index)

In [ ]:
text_data = df.patent_title_abstract.tolist()
text_data

In [ ]:
# partition data
len(text_data)
text_train = text_data[:round(len(text_data)*.8)]
text_test = text_data[round(len(text_data)*.8):]
print(len(text_data), len(text_train), len(text_test), len(text_train)+len(text_test))

### Pre-process text data

In [ ]:
# uncomment to download stop words from nltk and language package from spacy
# nltk.download('stopwords')
# nltk.download('punkt')
# !python -m spacy download en

In [ ]:
# construct pipeline using Spacy Language object and associated pipeline/components
nlp = spacy.load("en")
pprint(nlp.pipeline)

In [ ]:
processed_docs = []   

# process patent documents in pipeline
for doc in nlp.pipe(text_train, n_threads=4, batch_size=100):
   
    ents = doc.ents  # Named entities.

    # Keep only words (no numbers, no punctuation).
    # Lemmatize tokens, remove punctuation and remove stopwords.
    doc = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

    # Remove common words from a stopword list.
    doc = [token for token in doc if token not in stop_words]

    # Add named entities, but only if they are a compound of more than word.
    doc.extend([str(entity) for entity in ents if len(entity) > 1])
    
    processed_docs.append(doc)

processed_docs[0][:5]

In [ ]:
nlp.

In [ ]:
[token.text for token in doc]

In [ ]:
labels = set([w.label_ for w in doc.ents]) 

In [ ]:
for label in labels: 
    entities = [cleanup(e.string, lower=False) for e in document.ents if label==e.label_] 
    entities = list(set(entities)) 
    print(label,entities)

In [ ]:
pre_processed_docs = []
for doc in nlp.pipe(docs, n_threads=4, batch_size=100):
    # Process document using Spacy NLP pipeline.
    
    ents = doc.ents  # Named entities.

    # Keep only words (no numbers, no punctuation).
    # Lemmatize tokens, remove punctuation and remove stopwords.
    doc = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

    # Remove common words from a stopword list.
    #doc = [token for token in doc if token not in STOPWORDS]

    # Add named entities, but only if they are a compound of more than word.
    doc.extend([str(entity) for entity in ents if len(entity) > 1])
    
    pre_processed_docs.append(doc)

#### Tokenize

In [ ]:
# tokenize documents

def tokenize_docs(docs):
    tokenized_docs = []
    for doc in docs:
        tokenized_docs.append(word_tokenize(doc))
    return tokenized_docs

tokenized_docs = tokenize_docs(text_train)

#### Clean punctuation

In [ ]:
# clean punctuation
def clean_docs(tokenized_docs):
    clean_docs = []
    for doc in tokenized_docs:
       clean_docs.append([word for word in doc if word.isalpha()])  
    return clean_docs

In [ ]:
cleaned_data = clean_docs(tokenized_docs)
cleaned_data[0]

#### Convert to lowercase

In [ ]:
# convert to lowercase
def lower_words(docs):
    lowered_words = []
    for doc in docs:
        lowered_words.append([word.lower() for word in doc])
    return lowered_words

lowered_data = lower_words(cleaned_data)
lowered_data[0]

#### Clean stopwords

In [ ]:
# clean stopwords

stop_words = stopwords.words('english')

In [ ]:
def filter_stopwords(docs):
    filtered_docs = []
    for doc in docs:
       filtered_docs.append([word for word in doc if word not in stop_words])
    return filtered_docs

# remove stopwords
filtered_data = filter_stopwords(lowered_data)
filtered_data
# TODO (Lee) - resolve un-lowered stopwords "A" and "An", 'By', 'The'

#### Construct bigrams and trigrams

In [ ]:
# train bigram phrases model
bigram_model = Phrases(filtered_data, min_count=1, threshold=1)

# train trigram phrases model
trigram_model = Phrases(bigram_model[filtered_data], threshold=100)  

In [ ]:
# bigrams
def bigrams(docs):
    """create bigrams"""
    return [bigram_model[doc] for doc in docs]

In [ ]:
# initialize bigram and trigram models
bigram_model = gensim.models.phrases.Phraser(bigram_model)
trigram_model = gensim.models.phrases.Phraser(trigram_model)

In [ ]:
bigrams(filtered_data)[0]

In [ ]:
def trigrams(docs):
    """create trigrams"""
    return [trigram_model[bigram_model[doc]] for doc in docs]

In [ ]:
trigrams(filtered_data)[0]

#### Stem and Lemmatize

In [ ]:
def lemmatize_docs(docs, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """lemmatize documents"""
    lemmatized_docs = []
    for doc in docs: 
        lemmatized_docs.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return lemmatized_docs

In [ ]:
# TODO (Lee)

lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)
# for doc in cleaned_data:
#     for token in doc:
#         token.lemma_

# uncomment to use
# download english model with "python -m spacy download en"

# for token in doc:
#     print(token, token.lemma, token.lemma_)

# TODO (Lee) - lemmatize_docs(cleaned_data)

#### Create corpus and dictionary

In [ ]:
# using spacy pipeline components
# build dictionary
id_to_word = corpora.Dictionary(processed_docs)

# build corpus
texts = processed_docs

# apply term document frequency
# converts documents in corpus to bag-of-words format, a list of (token_id, token_count) tuples
corpus = [id_to_word.doc2bow(doc) for doc in processed_docs]

In [ ]:
 # build dictionary
id_to_word = corpora.Dictionary(filtered_data)

# build corpus
texts = filtered_data

# apply term document frequency
# converts documents in corpus to bag-of-words format, a list of (token_id, token_count) tuples
corpus = [id_to_word.doc2bow(text) for text in texts]

In [ ]:
# view formatted corpus (term-doc-frequency)
[[(id_to_word[id], freq) for id, freq in text] for text in corpus][:1]

### Model - model #1

In [ ]:
# TODO (Lee) - deprecation warnings
# construct LDA model
model_lda = LdaModel(corpus=corpus,
                     id2word=id_to_word,
                     num_topics=25, 
                     random_state=100,
                     update_every=1,
                     chunksize=100,
                     passes=10,
                     alpha='auto',
                     per_word_topics=True)

In [ ]:
# print keywords in n topics
pprint(model_lda.print_topics())

In [ ]:
# print top 10 keywords that comprise topic with index of 0
pprint(model_lda.print_topic(24))
# the most import keywords, and the respective weight, that form topic 0 are

In [ ]:
# print top 10 keywords that comprise topic with index of 1
pprint(model_lda.print_topic(1))

In [ ]:
# TODO (Lee) - infer topic from keywords?

### Evaluate - model #1

In [ ]:
# calculate perplexity metrics
perplexity = model_lda.log_perplexity(corpus)
perplexity

In [ ]:
# TODO (Lee) - confirm that filtered_data is indeed the correct dataset to pass to texts param
# calculate coherence metric
coherence = CoherenceModel(model=model_lda, texts=processed_docs, dictionary=id_to_word, coherence='c_v')
coherence_1 = coherence.get_coherence()
coherence_1

In [ ]:
# TODO (Lee) - confirm that filtered_data is indeed the correct dataset to pass to texts param
# calculate coherence metric
coherence = CoherenceModel(model=model_lda, texts=filtered_docs, dictionary=id_to_word, coherence='c_v')
coherence_1 = coherence.get_coherence()
coherence_1

In [ ]:
# calculate coherence metric or each of the n topicss
coherence_1 = coherence.get_coherence_per_topic()
coherence_1

In [ ]:
# explore topics
pyLDAvis.enable_notebook()
viz_topics_1 = pyLDAvis.gensim.prepare(model_lda, corpus, id_to_word)
viz_topics_1
# TODO (Lee) - salient vs relevant terms in pyLDA ?

### Model 2-  Mallet model

In [ ]:
# uncomment to download Mallet topic model
# !wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
# update this path
path_mallet = 'data/mallet-2.0.8/bin/mallet'

In [ ]:
model_2 = gensim.models.wrappers.LdaMallet(path_mallet, corpus=corpus, num_topics=25, id2word=id_to_word)

In [ ]:
# topics
pprint(model_2.show_topics(formatted=False))

In [ ]:
# calculate coherence metric
coherence_model_2 = CoherenceModel(model=model_2, texts=data, dictionary=id_to_word, coherence='c_v')
coherence_model_2 = coherence_model_2.get_coherence()
coherence_model_2

In [ ]:
# TODO (Lee)
# def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
#     """
#     Compute c_v coherence for various number of topics

#     Parameters:
#     ----------
#     dictionary : Gensim dictionary
#     corpus : Gensim corpus
#     texts : List of input texts
#     limit : Max num of topics

#     Returns:
#     -------
#     model_list : List of LDA topic models
#     coherence_values : Coherence values corresponding to the LDA model with respective number of topics
#     """
#     coherence_values = []
#     model_list = []
#     for num_topics in range(start, limit, step):
#         model = gensim.models.wrappers.LdaMallet(path_mallet, corpus=corpus, num_topics=num_topics, id2word=id2word)
#         model_list.append(model)
#         coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
#         coherence_values.append(coherencemodel.get_coherence())

#     return model_list, coherence_values

# model_list, coherence_values = compute_coherence_values(dictionary=id_to_word, corpus=corpus, texts=data, start=2, limit=40, step=6)

### Model 3 - Author topic model

In [ ]:
# construct inventor-to-doc mapping as df from nested inventors column in json api response
df_inventors = json_normalize(results['patents'], record_path=['inventors'], meta=['patent_number', 'patent_date'])
df_inventors = df_inventors[['inventor_id', 'patent_number', 'patent_date']]
df_inventors.sort_values(by=['patent_date'])
df_inventors.head(3)

In [ ]:
df.head(3)

In [ ]:
# TODO (Lee) - resolve workaround
df_idx = df
df_idx['idx'] = df.index
df_idx
df_idx_1 = df_idx[['patent_number', 'idx', 'inventors']]
df_idx_2 = df_idx_1.set_index('patent_number')
df_idx_2.pop('inventors')
df_idx_2
df_pat_idx = df_idx_2.T.to_dict('records')
for i in df_pat_idx:
    df_pat_idx = dict(i)
df_pat_idx

In [ ]:
df_pat_idx = df_idx_2.T.to_dict('records')
for i in df_pat_idx:
    df_pat_idx = dict(i)
df_pat_idx

In [ ]:
df_inv_test = json_normalize(results['patents'], record_path=['inventors'], meta=['patent_number', 'patent_date'])
df_inv_test.head(3)

In [ ]:
df_idx_pat_inv_map = df[['patent_number', 'inventors']]
df_idx_pat_inv_map.head(3)

In [ ]:
# TODO (Lee) - find out how to get list of patents_view_field names from API - I did it accidentally but need to replicate

In [ ]:
df.patent_title_abstract[0]

In [ ]:
df[:3]

In [ ]:
df_inventors.set_index('inventor_id').T.to_dict('list')

In [ ]:
# for k, v in pat2inv.items():
#     name_dict[new_key] = name_dict.pop(k)
#     time.sleep(4)

# pprint.pprint(name_dict)

# d = {'x':1, 'y':2, 'z':3}
# d1 = {'x':'a', 'y':'b', 'z':'c'}

# dict((d1[key], value) for (key, value) in d.items())
# {'a': 1, 'b': 2, 'c': 3}

In [ ]:
patdf2inv = dict((df_pat_idx[key], value) for (key, value) in pat2inv.items())
patdf2inv

In [ ]:
pat2inv = {k: list(v) for k,v in df_inventors.groupby("patent_number")["inventor_id"]}
pat2inv

In [ ]:
idx_pat_map = df.patent_number.to_dict()
idx_pat_map = {str(key): value for key, value in idx_pat_map.items()}
idx_pat_map

#### Construct author-topic model

In [ ]:
# construct author-topic model
model_at = AuthorTopicModel(corpus=corpus,
                         doc2author=patdf2inv,
                         id2word=id_to_word, 
                         num_topics=25)

In [ ]:
# construct vectors for authors
author_vecs = [model_at.get_author_topics(author) for author in model_at.id2author.values()]
author_vecs

In [ ]:
# retrieve the topic distribution for an author using use model[name] syntax
# each topic has a probability of being expressed given the particular author, but only the ones above a certain threshold are shown.

model_at['7788103-1']

In [ ]:
# def show_author(name):
#     print('\n%s' % name)
#     print('Docs:', model.author2doc[name])
#     print('Topics:')
#     pprint([(topic_labels[topic[0]], topic[1]) for topic in model[name]])

In [ ]:
# calculate per-word bound, which is a measure of the model's predictive performance (reconstruction error?)

build doc2author dictionary

doc2author = atmodel.construct_doc2author(model.corpus, model.author2doc)

In [ ]:
from gensim.models import atmodel
doc2author = atmodel.construct_doc2author(model.corpus, model.author2doc)

In [ ]:
gensim.models.atmodel.construct_author2doc(doc2author)
# construct mapping from author IDs to document IDs.

Parameters:	doc2author (dict of (int, list of str)) – Mapping of document id to authors.
Returns:	Mapping of authors to document ids.
Return type:	dict of (str, list of int)

In [ ]:
gensim.models.atmodel.construct_doc2author(corpus, author2doc)
construct mapping from document IDs to author IDs

Parameters:	
corpus (iterable of list of (int, float)) – Corpus in BoW format.
author2doc (dict of (str, list of int)) – Mapping of authors to documents.
Returns:	
Document to Author mapping.

Return type:	
dict of (int, list of str)